In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Reshape, BatchNormalization
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical

In [2]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [3]:
def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [4]:
def prepare_data(train_data, val_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    val_labels = encoder.transform(val_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [5]:
# import train data
adl_1_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("./full_dataset/CIP_interpolation/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("./full_dataset/CIP_interpolation/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("./full_dataset/CIP_interpolation/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S3.csv",header=None)

In [6]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
val_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [7]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 114), val (60949, 114), test (118750, 114)


In [8]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [9]:
print(scaled_train.shape)
print(train_labels.shape)

(497014, 113)
(497014, 18)


In [10]:
num_sensors = 113
window_size = 24
step_size = 12
classes = 18

In [11]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [12]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [41]:
model = Sequential()

size_of_kernel = (5,1)
kernel_strides = 1
num_filters = 64
num_lstm_cells = 128
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)
from keras.layers import PReLU

In [42]:
model.add(BatchNormalization(input_shape=inputshape))

In [43]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))
model.add(PReLU())

In [44]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='2_conv_layer'))
model.add(PReLU())

In [45]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='3_conv_layer'))
model.add(PReLU())

In [46]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='4_conv_layer'))
model.add(PReLU())

In [47]:
model.add(Reshape((8, num_filters*num_sensors)))

In [48]:
model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal', return_sequences=True, name='1_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal',return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(int(num_lstm_cells/2),kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer'))
model.add(Dropout(dropout_prob, name='4_dropout_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1),activation='softmax', name='softmax_layer'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [49]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

batch_normalization_7: input shape: (None, 24, 113, 1) output shape: (None, 24, 113, 1)
1_conv_layer: input shape: (None, 24, 113, 1) output shape: (None, 20, 113, 64)
p_re_lu_5: input shape: (None, 20, 113, 64) output shape: (None, 20, 113, 64)
2_conv_layer: input shape: (None, 20, 113, 64) output shape: (None, 16, 113, 64)
p_re_lu_6: input shape: (None, 16, 113, 64) output shape: (None, 16, 113, 64)
3_conv_layer: input shape: (None, 16, 113, 64) output shape: (None, 12, 113, 64)
p_re_lu_7: input shape: (None, 12, 113, 64) output shape: (None, 12, 113, 64)
4_conv_layer: input shape: (None, 12, 113, 64) output shape: (None, 8, 113, 64)
p_re_lu_8: input shape: (None, 8, 113, 64) output shape: (None, 8, 113, 64)
reshape_2: input shape: (None, 8, 113, 64) output shape: (None, 8, 7232)
1_lstm_layer: input shape: (None, 8, 7232) output shape: (None, 8, 128)
2_dropout_layer: input shape: (None, 8, 128) output shape: (None, 8, 128)
2_lstm_layer: input shape: (None, 8, 128) output shape: (None

In [50]:
batchSize = 200
train_epoches = 50

model.fit(reshaped_train,train_labels,validation_data=(reshaped_val,val_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)

train_epoches = 20
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_our_elaboration_full.h5')

Train on 41416 samples, validate on 5078 samples
Epoch 1/50
41416/41416 [==============================] - 71s 2ms/step - loss: 1.0680 - acc: 0.7245 - val_loss: 0.8704 - val_acc: 0.8304
Epoch 2/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.6556 - acc: 0.7887 - val_loss: 0.5432 - val_acc: 0.8452
Epoch 3/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.5171 - acc: 0.8244 - val_loss: 0.4858 - val_acc: 0.8557
Epoch 4/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.4412 - acc: 0.8472 - val_loss: 0.4818 - val_acc: 0.8637
Epoch 5/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.3859 - acc: 0.8635 - val_loss: 0.4890 - val_acc: 0.8637
Epoch 6/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.3476 - acc: 0.8754 - val_loss: 0.4470 - val_acc: 0.8694
Epoch 7/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.3205 - acc: 0.8843 - val_loss: 0.4633 - val_acc:

Epoch 9/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0359 - acc: 0.9898 - val_loss: 0.5471 - val_acc: 0.9118
Epoch 10/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0299 - acc: 0.9915 - val_loss: 0.6065 - val_acc: 0.9116
Epoch 11/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0332 - acc: 0.9911 - val_loss: 0.5402 - val_acc: 0.9151
Epoch 12/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0276 - acc: 0.9917 - val_loss: 0.6069 - val_acc: 0.9080
Epoch 13/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0299 - acc: 0.9913 - val_loss: 0.6670 - val_acc: 0.9032
Epoch 14/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0278 - acc: 0.9923 - val_loss: 0.6303 - val_acc: 0.9119
Epoch 15/50
46494/46494 [==============================] - 73s 2ms/step - loss: 0.0328 - acc: 0.9907 - val_loss: 0.6130 - val_acc: 0.9066
Epoch 16/50
46494/46494 [==========

In [51]:
predictions = model.predict(reshaped_test)

In [63]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

Standard accuracy is 0.9097432787548009
0 0.832524762482 0.957392571012
1 0.0058621386699 0.686131386861
2 0.00960177885587 0.815642458101
3 0.00606428138266 0.684210526316
4 0.00838892257934 0.850828729282
5 0.0230442692541 0.680952380952
6 0.0161714170204 0.755852842809
7 0.0101071356378 0.604166666667
8 0.00778249444108 0.567010309278
9 0.00394178289873 0.404494382022
10 0.00424499696786 0.475
11 0.0040428542551 0.421052631579
12 0.00262785526582 0.436363636364
13 0.0067717808773 0.615384615385
14 0.00616535273903 0.594594594595
15 0.0100060642814 0.655737704918
16 0.0320396199717 0.721461187215
17 0.0106124924196 0.693069306931
The weigths sum is 1.0
The computed f1-score is 0.909952812032923
The f1-score with sklearn function is 0.9099528120329231
Average f1-score is 0.6455192183492321


In [75]:
from keras.models import Model
from sklearn.decomposition import PCA

layer_name = 'dense_layer'
inter_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

network_output = inter_layer_model.predict(all_train)

# PCA with certain number of principal components
pca = PCA(n_components=40)
pca.fit(network_output)
new_output = pca.transform(network_output)

In [76]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

svm_labels = np.argmax(all_labels, axis=1)

# train a one-vs-one multi-class support vector machine
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(new_output, svm_labels)

# predict test data
svm_pred = clf.predict(pca.transform(inter_layer_model.predict(reshaped_test)))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

scores = cross_val_score(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

[ 0.91083123  0.92839133  0.92467139  0.90025316  0.89761784]
Accuracy: 0.91 (+/- 0.02)
Test accuracy is: 0.9108550636749545
The f1-score with sklearn function is 0.9090554906798731
Average f1-score is 0.6439139538765573


In [77]:
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(predicted, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(predicted, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, predicted, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

Test accuracy is: 0.9123711340206185
The f1-score with sklearn function is 0.9070462523665573
Average f1-score is 0.630115685381004


In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)